In [51]:
# Suprimir advertencias
import warnings
warnings.filterwarnings("ignore")


#Importing Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import datetime
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency

In [53]:
#Loading Data:
data = pd.read_csv("IFPTML-OD.csv", sep=';')
data.head(10)

,Type of data.,f_obs,prediction_trainig,f_ref,sIcI,sIcIV,sIIcIV,sIIIcII,sIVcII,sIVcIII,sVcIV,sVIcI,sVIIcII
0,Original,0,training,0.4167,0.171,0.737,1.102,0.000,0.000,0.000,0.967,0.000,0.593
1,Original,1,training,0.4167,0.010,1.240,0.000,0.000,1.200,1.533,1.615,0.000,0.000
2,Original,1,training,0.4167,0.244,1.647,0.677,0.632,0.000,0.000,1.164,0.000,0.000
3,Original,1,validation,0.4167,0.306,1.190,0.723,0.818,0.788,1.034,1.153,0.000,0.887
4,Original,1,training,0.4167,0.069,0.521,1.004,0.000,0.348,1.094,0.471,0.128,0.478
5,Original,0,training,0.4167,0.244,0.606,0.328,0.221,0.084,0.857,0.164,0.139,0.162
6,Original,1,training,0.4167,0.592,1.302,0.677,0.000,0.000,0.000,1.027,0.540,0.077
7,Original,0,validation,0.4167,0.250,1.045,0.451,0.000,0.005,0.242,0.306,0.000,0.175
8,Original,1,training,0.4167,0.011,0.720,0.000,0.048,0.000,0.000,0.832,0.000,0.284
9,Original,1,training,0.4167,0.450,0.519,0.315,0.169,0.036,0.032,0.368,0.581,0.144


In [54]:
#Data Splitting:
Y = data.iloc[:, 1]
X = data.iloc[:, 3:]

In [55]:
#Splitting Data into Training and Prediction Sets:
train_data = data[data["prediction_trainig"] == "training"]
pred_data = data[data["prediction_trainig"] == "validation"]

In [56]:
# Model Preparation for LDA
X_train_LDA= train_data.iloc[:, 3:]
X_test_LDA= pred_data.iloc[:, 3:]
y_train_LDA= train_data.iloc[:, 1]
y_test_LDA= pred_data.iloc[:, 1]

print(X_train_LDA.shape)
print(X_test_LDA.shape)
print(y_train_LDA.shape)
print(y_test_LDA.shape)

(436, 10)
(131, 10)
(436,)
(131,)


In [57]:
#X_LDA
mlr_X_train_LDA= X_train_LDA[['f_ref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIVcII', 'sIVcIII','sVcIV','sVIcI','sVIIcII']]
mlr_X_test_LDA= X_test_LDA[['f_ref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIVcII', 'sIVcIII','sVcIV','sVIcI','sVIIcII']]

In [58]:
#Model_LDA

In [59]:
# Define the Prior Classification Probabilities
priors = [0.5, 0.5]  

In [60]:
# Initialize the LDA model with the prior probabilities
LDA = LinearDiscriminantAnalysis(priors=priors)

In [61]:
#Fit the model

In [62]:
LDA.fit(mlr_X_train_LDA, y_train_LDA)

LinearDiscriminantAnalysis(priors=[0.5, 0.5])

In [63]:
y_pred_train_LDA = LDA.predict(mlr_X_train_LDA)
y_pred_test_LDA = LDA.predict(mlr_X_test_LDA)

In [64]:
#Perform cross-validation with LinearDiscriminantAnalysis
cv_scores = cross_val_score(LDA, mlr_X_train_LDA, y_train_LDA, cv=5)  # 5-fold cross-validation
print("Cross-validation Scores:", cv_scores)

Cross-validation Scores: [0.48863636 0.73563218 0.81609195 0.86206897 0.88505747]


In [65]:
# Get variable names
variable_names = X.columns

In [66]:
# Coefficients of the linear discriminant function
coefficients = LDA.coef_[0]
intercept = LDA.intercept_[0]

In [67]:
# Print the equation of the line with variable names
print("Equation of the line:")
print("y = ", end="")
for i, coefficient in enumerate(coefficients):
    variable_name = variable_names[i]
    print(f"({coefficient:.2f} * {variable_name}) + ", end="")
print(f"{intercept:.2f}")

Equation of the line:
y = (12.25 * f_ref) + (-1.31 * sIcI) + (4.00 * sIcIV) + (-1.73 * sIIcIV) + (0.28 * sIIIcII) + (-0.60 * sIVcII) + (-0.05 * sIVcIII) + (1.99 * sVcIV) + (1.06 * sVIcI) + (-0.17 * sVIIcII) + -5.41


In [68]:
confusion_train = confusion_matrix(y_train_LDA,y_pred_train_LDA)
confusion_test = confusion_matrix(y_test_LDA, y_pred_test_LDA)

In [69]:
y_train_pred_proba = LDA.predict_proba(mlr_X_train_LDA)
y_test_pred_proba = LDA.predict_proba(mlr_X_test_LDA)

In [70]:
print('Confusion Matrix for Training Series')
print(confusion_train)
print('Confusion Matrix for Test Series')
print(confusion_test)

Confusion Matrix for Training Series
[[286  61]
 [ 19  70]]
Confusion Matrix for Test Series
[[91 21]
 [ 3 16]]


In [71]:
tn, fp, fn, tp = confusion_matrix(y_train_LDA, y_pred_train_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [72]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy)

Sp(%)= 0.8242074927953891 Sn(%)= 0.7865168539325843 Ac(%)= 0.8165137614678899


In [73]:
tn, fp, fn, tp = confusion_matrix(y_test_LDA, y_pred_test_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [74]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy),

Sp(%)= 0.8125 Sn(%)= 0.8421052631578947 Ac(%)= 0.816793893129771


(None,)

In [75]:
# Calculations for IFPTML model validation  f(vij)pred

In [76]:
#Loading Data:
data = pd.read_csv("IFPTML Validation.csv", sep=';')
data.head(10)

,f_ref,sIcI,sIcIV,sIIcIV,sIIIcII,sIVcII,sIVcIII,sVcIV,sVIcI,sVIIcII
0,0.417,0.3709,1.0150,0.3281,0.024679,0.2358,0.1490,0.3369,0.333368,0.117878
1,0.417,0.2001,1.1944,1.0734,1.000304,1.0088,0.0006,0.6776,0.107775,0.926115
2,0.417,0.2578,1.0364,0.7026,0.313302,0.3133,0.1431,0.5755,0.067218,0.250313
3,0.417,0.3891,1.0040,0.6427,0.029167,0.0821,0.4494,0.1645,0.560819,0.098112
4,0.417,0.4962,0.9027,0.3281,0.024679,0.2176,0.4433,0.3369,0.257195,0.117878
5,0.417,0.2437,0.5449,0.8144,0.024679,0.0580,0.3311,0.3301,0.281990,0.117878
6,0.417,0.1971,0.5449,0.8144,0.024679,0.2358,0.1488,0.2970,0.228431,0.117878
7,0.417,0.3133,0.5649,0.2997,0.284592,0.0580,0.0878,0.6334,0.310107,0.117878
8,0.417,0.6050,1.2385,0.6427,0.540705,0.4559,0.7373,0.1645,0.633936,0.508218
9,0.417,0.3740,1.5064,1.7655,0.344602,0.3257,0.2645,1.0618,0.345676,0.207798


In [49]:
print(data.columns)

Index(['f_ref', 'sIcI', 'sIcIV', 'sIIcIV', 'sIIIcII', 'sIVcII', 'sIVcIII',
       'sVcIV', 'sVIcI', 'sVIIcII'],
      dtype='object')


In [50]:
X_prediction = data.iloc[:, 0:]
LDA_X_prediction= X_prediction[['f_ref', 'sIcI', 'sIcIV', 'sIIcIV', 'sIIIcII', 'sIVcII', 'sIVcIII',
       'sVcIV', 'sVIcI', 'sVIIcII']]

In [81]:
X_prediction.head(6)

,f_ref,sIcI,sIcIV,sIIcIV,sIIIcII,sIVcII,sIVcIII,sVcIV,sVIcI,sVIIcII
0,0.417,0.3709,1.0150,0.3281,0.024679,0.2358,0.1490,0.3369,0.333368,0.117878
1,0.417,0.2001,1.1944,1.0734,1.000304,1.0088,0.0006,0.6776,0.107775,0.926115
2,0.417,0.2578,1.0364,0.7026,0.313302,0.3133,0.1431,0.5755,0.067218,0.250313
3,0.417,0.3891,1.0040,0.6427,0.029167,0.0821,0.4494,0.1645,0.560819,0.098112
4,0.417,0.4962,0.9027,0.3281,0.024679,0.2176,0.4433,0.3369,0.257195,0.117878
5,0.417,0.2437,0.5449,0.8144,0.024679,0.0580,0.3311,0.3301,0.281990,0.117878


In [87]:
# Class prediction
y_pred_LDA = LDA.predict(LDA_X_prediction)

In [88]:
# Predict the class membership probabilities for each sample
y_pred_proba_LDA = LDA.predict_proba(LDA_X_prediction)

In [89]:
# Create a DataFrame from the class predictions
df = pd.DataFrame(y_pred_LDA)

In [90]:
# Define the file path where you want to save the CSV file
file_path = "pred_LDA_20.csv"

In [91]:
# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"Predictions saved to {file_path}")

Predictions saved to pred_LDA_20.csv


In [93]:
df = pd.DataFrame(y_pred_proba_LDA)

# Define the file path where you want to save the CSV file
file_path = "pred_LDA_21.csv"

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"Predicted saved to {file_path}")

Predicted saved to pred_LDA_21.csv
